In [1]:
import cirq
import openfermion
import numpy as np
import datetime
# from anzats import Anzats
# from expectation import get_expectation_ZiZj, get_expectation_ghz_l4, get_expectation_ghz_l8
# from optimization import get_gradient, optimize_by_gradient_descent
Pi=3.1415


In [20]:
from functools import partial

## anzats

In [2]:
class Anzats():
    def __init__(self, length, gamma, beta):

        # initialize circuit
        circuit = cirq.Circuit()
        qubits = cirq.LineQubit.range(length)

        for i in range(length):
            circuit.append(cirq.H(qubits[i]))

        # add gamma circuit
        for index in range(gamma.size):
            # add gates on 2n and 2n+1
            for i in range(1, length, 2):
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
                    )
                # circuit.append(
                #     cirq.ZZPowGate(
                #         exponent=gamma[index]/Pi, global_shift=0.0).on(
                #             qubits[i], qubits[(i+1)%length]
                #     )
                # )
            # add gates on 2n-1 and 2n
            for i in range(0, length, 2):
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
                    )
                # circuit.append(
                #     cirq.ZZPowGate(
                #         exponent=gamma[index]/Pi, global_shift=0.0).on(
                #             qubits[i], qubits[(i+1)%length]
                #         )
                # )

            # add beta circuit
            for i in range(length):
                circuit.append(
                    cirq.XPowGate(
                        exponent=beta[index]*2/Pi, global_shift=0.0).on(
                        qubits[i])
                )

        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta




## expectation

In [3]:
class CriticalStateArgs():
    def __init__(self, length, g):
        self.length = length
        self.g = g


def get_expectation_critical_state(function_args, gamma, beta):
    anzats = Anzats(function_args.length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    vector = cirq.final_state_vector(circuit)

    value = 0 + 0j
    for i in range(function_args.length):
        circuit = anzats.circuit.copy()
        circuit.append(cirq.Z(qubits[i]))
        circuit.append(cirq.Z(qubits[(i+1)%function_args.length]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    for i in range(function_args.length):
        circuit = anzats.circuit.copy()
        circuit.append(function_args.g * cirq.X(qubits[i]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    return value

# def get_expectation_critical_state_l4(gamma: np.array, beta: np.array):
#     return get_expectation_critical_state(4, gamma, beta)

# def get_expectation_critical_state_l8(gamma: np.array, beta: np.array):
#     return get_expectation_critical_state(8, gamma, beta)

# def get_expectation_critical_state_l10(gamma: np.array, beta: np.array):
#     return get_expectation_critical_state(10, gamma, beta)

# def get_expectation_critical_state_l12(gamma: np.array, beta: np.array):
#     return get_expectation_critical_state(12, gamma, beta)

# def get_expectation_critical_state_l14(gamma: np.array, beta: np.array):
#     return get_expectation_critical_state(14, gamma, beta)

# def get_expectation_critical_state_l16(gamma: np.array, beta: np.array):
#     return get_expectation_critical_state(16, gamma, beta)


def get_expectation_ghz(length, gamma, beta):
    anzats = Anzats(length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    vector = cirq.final_state_vector(circuit)

    value = 0 + 0j
    for i in range(length):
        circuit = anzats.circuit.copy()
        circuit.append(cirq.Z(qubits[i]))
        circuit.append(cirq.Z(qubits[(i+1)%length]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    return value


def get_expectation_ghz_l4(gamma: np.array, beta: np.array):
    return get_expectation_ghz(4, gamma, beta)

def get_expectation_ghz_l8(gamma: np.array, beta: np.array):
    return get_expectation_ghz(8, gamma, beta)

def get_expectation_ghz_l10(gamma: np.array, beta: np.array):
    return get_expectation_ghz(10, gamma, beta)

def get_expectation_ghz_l12(gamma: np.array, beta: np.array):
    return get_expectation_ghz(12, gamma, beta)

def get_expectation_ghz_l14(gamma: np.array, beta: np.array):
    return get_expectation_ghz(14, gamma, beta)

def get_expectation_ghz_l16(gamma: np.array, beta: np.array):
    return get_expectation_ghz(16, gamma, beta)


## optimization

In [26]:
def get_gradient(function, gamma: np.array, beta: np.array, delta_gamma, delta_beta, iter):
    grad_gamma = np.zeros_like(gamma)
    grad_beta  = np.zeros_like(beta)
    gamma_edge = gamma
    beta_edge  = beta
    # initial gamma, beta?
    
    if not (gamma.size == beta.size):
        return 1

    for index in range(gamma.size):
        center = gamma[index]
        gamma_edge[index] = gamma[index] - delta_gamma
        e1 = function(gamma=gamma_edge, beta=beta)
        gamma_edge[index] = gamma[index] + delta_gamma
        e2 = function(gamma=gamma_edge, beta=beta)
        grad_gamma[index] = (e2.real-e1.real)/(2*delta_gamma)
        gamma[index] = center

        center = beta[index]
        beta_edge[index] = beta[index] - delta_beta
        e1 = function(gamma=gamma, beta=beta_edge)
        beta_edge[index] = beta[index] + delta_beta
        e2 = function(gamma=gamma, beta=beta_edge)
        grad_beta[index] = (e2.real-e1.real)/(2*delta_beta)
        beta[index] = center
    
    return grad_gamma, grad_beta


def optimize_by_gradient_descent(function, initial_gamma: np.array, initial_beta: np.array, alpha, delta_gamma, delta_beta, iteration, Figure=True,filepath=""):
    gamma, beta = initial_gamma, initial_beta

    textlines = []
    textlines.append("iter, energy, gamma[0], beta[0],...")
    print("iter, energy, gamma[0], beta[0],...")

    for iter in range(int(iteration)):
        # it is complex for me to set get_gradient for two optical parameter_vector
        grad_gamma, grad_beta = get_gradient(function, gamma, beta, delta_gamma, delta_beta, iter)
        gamma -= alpha * grad_gamma
        beta  -= alpha * grad_beta
        energy = function(gamma=gamma, beta=beta)

        record = [iter, energy]
        for index in range(gamma.size):
            record.append(gamma[index])
            record.append(beta[index])
        textlines.append(record)
        print(record)
    
    if len(filepath)>0:
        with open(filepath, mode='a') as f:
            for i, textline in enumerate(textlines):
                f.write("{}\n".format(textline))

    return gamma, beta

def get_gradient_with_args(function, function_args, gamma: np.array, beta: np.array, delta_gamma, delta_beta, iter):
    grad_gamma = np.zeros_like(gamma)
    grad_beta  = np.zeros_like(beta)
    gamma_edge = gamma
    beta_edge  = beta
    # initial gamma, beta?
    
    if not (gamma.size == beta.size):
        return 1

    for index in range(gamma.size):
        center = gamma[index]
        gamma_edge[index] = gamma[index] - delta_gamma
        e1 = function(function_args, gamma_edge, beta)
        gamma_edge[index] = gamma[index] + delta_gamma
        e2 = function(function_args, gamma_edge, beta)
        grad_gamma[index] = (e2.real-e1.real)/(2*delta_gamma)
        gamma[index] = center

        center = beta[index]
        beta_edge[index] = beta[index] - delta_beta
        e1 = function(function_args, gamma, beta_edge)
        beta_edge[index] = beta[index] + delta_beta
        e2 = function(function_args, gamma, beta_edge)
        grad_beta[index] = (e2.real-e1.real)/(2*delta_beta)
        beta[index] = center
    
    return grad_gamma, grad_beta

def optimize_by_gradient_descent_with_args(function, function_args, initial_gamma: np.array, initial_beta: np.array, alpha, delta_gamma, delta_beta, iteration, Figure=True,filepath=""):
    gamma, beta = initial_gamma, initial_beta

    textlines = []
    textlines.append("iter, energy, gamma[0], beta[0],...")
    print("iter, energy, gamma[0], beta[0],...")

    for iter in range(int(iteration)):
        # it is complex for me to set get_gradient for two optical parameter_vector
        grad_gamma, grad_beta = get_gradient_with_args(function, function_args, gamma, beta, delta_gamma, delta_beta, iter)
        gamma -= alpha * grad_gamma
        beta  -= alpha * grad_beta
        energy = function(function_args, gamma, beta)

        record = [iter, energy]
        for index in range(gamma.size):
            record.append(gamma[index])
            record.append(beta[index])
        textlines.append(record)
        print(record)
    
    if len(filepath)>0:
        with open(filepath, mode='a') as f:
            for i, textline in enumerate(textlines):
                f.write("{}\n".format(textline))

    return gamma, beta

## case: length 4, static gamma and beta

In [6]:
length = 4
g = 1.0
gamma = np.array([0.5, 0.4, 0.3, 0.2])
beta  = np.array([0.7, 0.5, 0.3, 0.1])
function_args = CriticalStateArgs(length, g)

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
# print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_critical_state(function_args,gamma, beta)
print("expectation on ghz: {}".format(value))

measurements: (no measurements)

qubits: (cirq.LineQubit(3), cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2))
output vector: [ 0.21862863-0.3136721j   0.01913893-0.24795502j  0.01913894-0.24795511j
  0.00876991-0.07031819j  0.01913892-0.24795505j -0.1374358 -0.278366j
  0.00876982-0.0703182j   0.01913891-0.24795498j  0.01913891-0.24795498j
  0.00876982-0.0703182j  -0.1374358 -0.278366j    0.01913892-0.24795505j
  0.00876991-0.07031819j  0.01913894-0.24795511j  0.01913893-0.24795502j
  0.21862863-0.3136721j ]

phase:
output vector: |⟩
norm: (1+0j)
expectation on ghz: (-3.335458993911743+0j)


## case: length 8, static gamma and beta (paper value)

In [14]:
length = 8
gamma = np.array([0.2496, 0.4808, 0.5260, 0.4503])
beta  = np.array([0.6845, 0.6559, 0.6048, 0.3180])
g = 1.0
function_args = CriticalStateArgs(length, g)

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
# print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_critical_state(function_args, gamma, beta)
print("expectation on ghz: {}".format(value))

measurements: (no measurements)

qubits: (cirq.LineQubit(7), cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4), cirq.LineQubit(5), cirq.LineQubit(6))
output vector: [0.4591058 -4.49337065e-03j 0.12919495-1.25825033e-03j
 0.12919495-1.25830155e-03j 0.07737847-7.35415146e-04j
 0.12919495-1.25823729e-03j 0.04038734-3.95270996e-04j
 0.07737848-7.35405833e-04j 0.06066925-5.82892448e-04j
 0.12919493-1.25823729e-03j 0.03783053-3.70066147e-04j
 0.04038735-3.95267271e-04j 0.02739703-2.70092394e-04j
 0.0773785 -7.35389534e-04j 0.02739705-2.70094723e-04j
 0.06066927-5.82899433e-04j 0.05631589-5.69676980e-04j
 0.12919495-1.25824474e-03j 0.03746548-3.55740078e-04j
 0.03783052-3.70071735e-04j 0.0245581 -2.27298122e-04j
 0.04038734-3.95286363e-04j 0.01372327-1.31525332e-04j
 0.02739706-2.70090764e-04j 0.02465377-2.33101659e-04j
 0.0773785 -7.35399313e-04j 0.02455809-2.27314420e-04j
 0.02739705-2.70107295e-04j 0.02209693-2.07863282e-04j
 0.06066924-5.8288779

## case: length 4, optimize beta and gamma

In [27]:
length = 4
initial_gamma = np.array([0.1, 0.2])
initial_beta  = np.array([0.1, 0.2])
g = 1.0
function_args = CriticalStateArgs(length, g)

iteration = 10
alpha = 0.1
delta_gamma = 0.001
delta_beta  = 0.001

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_critical_state, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)
# gamma, beta = optimize_by_gradient_descent_with_args(get_expectation_critical_state, function_args, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

iter, energy, gamma[0], beta[0],...
[0, (-3.3336696922779083+0j), -0.04722943305969238, 0.2243501901626587, -0.04554580450057982, 0.4212628722190857]
[1, (-3.120816722512245+0j), 0.27403289675712583, 0.2961812376976013, 0.4582736313343048, 0.4917975187301636]
[2, (-4.527279317378998+0j), 0.22860745191574094, 0.776504522562027, 0.25659908056259156, 0.389121812582016]
[3, (-2.3046870827674866+0j), 0.0016895890235900657, 0.6361996591091156, 0.5639467060565948, 0.3858457922935486]
[4, (0.5396678447723389+0j), 0.6504325032234192, 0.6365125834941864, 0.0022094368934630904, 0.3947276294231415]
[5, (0.7601370513439178+0j), 0.7273061037063598, 0.5223786294460296, 0.7178220570087432, 0.28230656385421754]
[6, (-3.8776921927928925+1.862645149230957e-09j), 0.12693682909011839, -0.052266770601272605, 0.27294788956642146, 0.4055421471595764]
[7, (0.49438080191612244+0j), -0.31533857583999636, -0.0043461263179779275, -0.1535382449626923, 0.41068602800369264]
[8, (-4.326667338609695+0j), -0.05661567449

## case: length 8, optimize beta and gamma

In [12]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
g = 1.0
function_args = CriticalStateArgs(length, g)

iteration = 10
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

print("exact energy: ")
print(-1.28*length)

gamma, beta = optimize_by_gradient_descent_with_args(get_expectation_critical_state, function_args, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

iter, energy, gamma[0], beta[0],...
[0, (-8.643898367881775-1.4901161193847656e-08j), 0.459535151720047, 0.5285045802593231, 0.5193509459495544, 0.4975709617137909, 0.5042518973350525, 0.5230561196804047, 0.4914152920246124, 0.5004073977470398]
[1, (-9.043593496084213+2.9802322387695312e-08j), 0.4267871677875519, 0.5465790927410126, 0.5324384868144989, 0.4990096688270569, 0.5084339082241058, 0.5437770783901215, 0.4794410169124603, 0.4914412200450897]
[2, (-9.293250501155853+4.470348358154297e-08j), 0.40175873041152954, 0.5584051012992859, 0.5401006639003754, 0.5057467818260193, 0.5132672488689423, 0.5595863163471222, 0.4691413342952728, 0.47944891452789307]
[3, (-9.461697429418564-1.4901161193847656e-08j), 0.38322019577026367, 0.567211389541626, 0.543816864490509, 0.516845315694809, 0.5188052654266357, 0.5710107386112213, 0.46224671602249146, 0.4674724042415619]
[4, (-9.592488467693329+4.470348358154297e-08j), 0.3692628741264343, 0.575039267539978, 0.545166015625, 0.5303232669830322, 0

## case: length 8, optimize beta and gamma b

In [13]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
g = 1.0
function_args = CriticalStateArgs(length, g)

iteration = 20
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

print("energy in paper: ")
print(-1.28*length)

gamma, beta = optimize_by_gradient_descent_with_args(get_expectation_critical_state, function_args, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

exact energy: 
-10.24
iter, energy, gamma[0], beta[0],...
[0, (-8.643898367881775-1.4901161193847656e-08j), 0.459535151720047, 0.5285045802593231, 0.5193509459495544, 0.4975709617137909, 0.5042518973350525, 0.5230561196804047, 0.4914152920246124, 0.5004073977470398]
[1, (-9.043593496084213+2.9802322387695312e-08j), 0.4267871677875519, 0.5465790927410126, 0.5324384868144989, 0.4990096688270569, 0.5084339082241058, 0.5437770783901215, 0.4794410169124603, 0.4914412200450897]
[2, (-9.293250501155853+4.470348358154297e-08j), 0.40175873041152954, 0.5584051012992859, 0.5401006639003754, 0.5057467818260193, 0.5132672488689423, 0.5595863163471222, 0.4691413342952728, 0.47944891452789307]
[3, (-9.461697429418564-1.4901161193847656e-08j), 0.38322019577026367, 0.567211389541626, 0.543816864490509, 0.516845315694809, 0.5188052654266357, 0.5710107386112213, 0.46224671602249146, 0.4674724042415619]
[4, (-9.592488467693329+4.470348358154297e-08j), 0.3692628741264343, 0.575039267539978, 0.545166015625,

## case: length 16, static beta and gamma (paper value)

In [18]:
length = 16
p = 8
initial_gamma = np.array([0.3790,0.5638,0.9046,0.6738,0.8616,0.5624,0.5224,0.4119])
initial_beta  = np.array([0.5622,0.7101,0.3210,0.8377,0.4004,0.9450,0.6466,0.5172])
g = 1.0
function_args = CriticalStateArgs(length, g)


print("energy in paper: ")
print(-1.275*length)
print(get_expectation_critical_state(function_args, initial_gamma, initial_beta))

# iteration = 100
# alpha = 0.1
# delta_gamma = 0.001
# delta_beta  = 0.001

# gamma, beta = optimize_by_gradient_descent(get_expectation_ghz_l16, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

energy in paper: 
-20.4
(-20.404566824436188+1.4901161193847656e-08j)
[0.2496 0.4808 0.526  0.4503] [0.6845 0.6559 0.6048 0.318 ]


## playground

In [ ]:
L = 10
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(L)

for i in range(1, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
        )
    )


for i in range(2, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
            )
    )


for i in range(L):
    circuit.append(
        cirq.XPowGate(
            exponent=0.5, global_shift=0.0).on(
            qubits[i])
    )


# for i in range(1, L, 2):
#     circuit.append(cirq.Z(qubits[i-1]))
#     circuit.append(cirq.Z(qubits[i]))


# for i in range(2, L, 2):
#     circuit.append(cirq.Y(qubits[i-1]))
#     circuit.append(cirq.Y(qubits[i]))


# for i in range(L):
#     circuit.append(cirq.X(qubits[i]))



print(circuit)


0: ───ZZ───────X^0.5────────────
      │
1: ───ZZ^0.5───ZZ───────X^0.5───
               │
2: ───ZZ───────ZZ^0.5───X^0.5───
      │
3: ───ZZ^0.5───ZZ───────X^0.5───
               │
4: ───ZZ───────ZZ^0.5───X^0.5───
      │
5: ───ZZ^0.5───ZZ───────X^0.5───
               │
6: ───ZZ───────ZZ^0.5───X^0.5───
      │
7: ───ZZ^0.5───ZZ───────X^0.5───
               │
8: ───ZZ───────ZZ^0.5───X^0.5───
      │
9: ───ZZ^0.5───X^0.5────────────


In [ ]:
value = 0 + 0j

# circuit2 = circuit
# anzats2 = Anzats(length, gamma, beta)
# circuit2 = anzats2.circuit
# qubits = anzats2.qubits
# circuit2.append(
#         cirq.ZZPowGate(
#             exponent=1, global_shift=0.0).on(
#                 qubits[1], qubits[2]
#         ))
# print(circuit2)
# result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
# vector2 = cirq.final_state_vector(circuit2)
# value -= np.dot(vector2.conjugate(), vector)

# print(value)

# # for i in range(length-1):
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[i], qubits[i+1]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)
# else:
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[length-1], qubits[0]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)

In [ ]:
a = np.array([1,3])
a[1]

3

In [ ]:
print(vector)

[0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.062499

In [ ]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

p = 1
initial_gamma = np.array([-3.0 for i in range(p)])
initial_beta  = np.array([4.0 for i in range(p)])

alpha = 0.1
delta_gamma = 0.0001
delta_beta = 0.0001
iteration = 100

final_gamma, final_beta = optimize_by_gradient_descent(function_norm, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration, Figure=True)

print(final_gamma, final_beta)

iter, energy, gamma[0], beta[0],...
[0, 20.250009000048443, -2.6999950000002855, 3.6000049999995696]
[1, 16.402515390162296, -2.4299904999994766, 3.2400094999975835]
[2, 13.286044756250423, -2.186986449998926, 2.9160135499979276]
[3, 10.761702813850011, -1.9682828049986085, 2.6244171949971715]
[4, 8.71698518447511, -1.771449524498145, 2.3619804754972904]
[5, 7.060763314251251, -1.5942995720487048, 2.1257874279467757]
[6, 5.71922306798135, -1.4348646148437645, 1.9132136851514012]
[7, 4.632574990255437, -1.2913731533593626, 1.721897316635964]
[8, 3.7523896168740816, -1.1622308380235786, 1.5497125849721591]
[9, 3.039439077053899, -1.0460027542212842, 1.3947463264749569]
[10, 2.461948791156269, -0.9413974787991748, 1.2552766938276303]
[11, 1.9941813457998523, -0.8472527309192319, 1.1297540244450364]
[12, 1.6152894326596015, -0.7625224578273722, 1.0167836220005242]
[13, 1.3083867288548008, -0.6862652120446207, 0.9151102598005076]
[14, 1.0597953100277273, -0.617633690840111, 0.82360423382043

In [ ]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

initial_gamma = np.array([3.0, 2.0])
initial_beta = np.array([4.0, 5.0])

final_gamma, final_beta = optimize_by_gradient_descent(
    function=function_norm,
    initial_gamma=initial_gamma,
    initial_beta=initial_beta,
    alpha=0.1,
    delta_gamma=0.001,
    delta_beta=0.001,
    iteration=100,
    Figure=True)
print(final_gamma, final_beta)
print(function_norm(gamma, beta))

iter, energy, gamma[0], beta[0],...
[0, 43.741260009996445, 2.7000499999998624, 3.6000500000000955, 1.8000499999999846, 4.500049999999618]
[1, 35.43155463610003, 2.4300949999999517, 3.2400950000003945, 1.6200949999998642, 4.0500949999997715]
[2, 28.700579899441102, 2.1871354999997124, 2.9161355000003795, 1.4581354999997558, 3.6451354999999808]
[3, 23.248388317329418, 1.9684719499996035, 2.6245719500004725, 1.3123719499998003, 3.2806719500002757]
[4, 18.83202129493629, 1.7716747549995944, 2.3621647550002365, 1.1811847549998404, 2.9526547549999904]
[5, 15.254681350009095, 1.5945572794996394, 2.1259982795000596, 1.0631162794997522, 2.657439279499947]
[6, 12.35696160350955, 1.4351515515496267, 1.9134484515502557, 0.9568546515497083, 2.391745351550174]
[7, 10.00974165684386, 1.2916863963946952, 1.7221536063953344, 0.8612191863947665, 2.1526208163951743]
[8, 8.108433243244505, 1.162567756755248, 1.5499882457558698, 0.7751472677553366, 1.9374087347556923]
[9, 6.568319197107911, 1.046360981079

In [ ]:
length = 6
gamma = np.array([0.1, 0.1])
beta = np.array([0.1, 0.1])

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
qubits = anzats.qubits
vector = cirq.final_state_vector(circuit)

ham_nd = np.zeros((length, length), dtype=np.complex128)

print(cirq.Z(qubits[-1]))
for i in range(length-1):
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[i]))
    circuit_ZiZj.append(cirq.Z(qubits[i+1]))
    ham_nd += cirq.unitary(circuit_ZiZj)
else:
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[length-1]))
    circuit_ZiZj.append(cirq.Z(qubits[0]))
    ham_nd += cirq.unitary(circuit_ZiZj)
    
print(ham_nd)


# def get_expectation_ZiZj(length, beta, gamma, qubit_ZZ_on_i, qubit_ZZ_on_j, print_circuit=False):
#     """
#     controls of append on circuit are accumulated.
#     we have to make circuits on each hamiltonian terms.
#     """
#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     vector = cirq.final_state_vector(circuit)

#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_i]))
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_j]))
#     if print_circuit:
#         print(circuit)
#     simulator = cirq.Simulator()
#     # result2 = simulator.simulate(circuit, qubit_order=qubits)
#     vector2 = cirq.final_state_vector(circuit)
#     return np.dot(vector2.conjugate(), vector)


# def get_expectation_ghz_l4(gamma: np.array, beta: np.array):
#     length = 4
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


# def get_expectation_ghz_l8(gamma: np.array, beta: np.array):
#     length = 8
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


Z(q(5))


ValueError: operands could not be broadcast together with shapes (6,6) (4,4) (6,6) 